<a href="https://colab.research.google.com/github/vaTomas/lyric-video/blob/notebooks/notebooks/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Test

In [16]:
!git clone https://github.com/vaTomas/lyric-video.git

fatal: destination path 'lyric-video' already exists and is not an empty directory.


## Install Dependencies

In [46]:
%pip install '/content/lyric-video/'

Processing ./lyric-video
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 48.9 MB/s eta 0:00:00
  Created wheel for lyric_video: filename=lyric_video-0.0.1-py3-none-any.whl size=60296 sha256=bed1b28fe989d2c18ae4871959f99e3fb2efe83f77ca316b44281046dc98d96e
  Stored in directory: /root/.cache/pip/wheels/ed/a3/49/928ad5130b6c00e21189851452efffdb22b468754b1f83f772
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22427 sha256=d3d0b447f99b879e54f976ba4d72cbd8145091ecb6eb32687ae765ca27be5164
  Stored in directory: /root/.cache/pip/wheels/1f/43/f1/23ee9119497fcb57d9f7046fbf34c6d9027c46a1fa7824cf08
Successfully built lyric_video srt
  Attempting uninstall: pillow
    Found existing installation: pillow 11.2.1
    Uninsta

In [45]:
import sys

src_path = '/content/lyric-video/src'

if src_path not in sys.path:
    sys.path.append(src_path)
    print(f"Added '{src_path}' to sys.path")
else:
    print(f"'{src_path}' is already in sys.path")

'/content/lyric-video/src' is already in sys.path


In [40]:
from subtitle import get_subtitles

ModuleNotFoundError: No module named 'srt'